## Problem Set 3 (Part 2 of 2): Interacting with the Alpha Vantage API using Python libraries

In Part 2 of Problem Set 3 we will be using the data retrived from the Alpha Vantage API and creating an optimized portfolio of stocks based on Modern Portfolio Theory. The PyPortfolioOpt package will be used to optimize a portfolio of stocks based maximizing the Sharpe Ratio. This exercise is an example of how a robo-advisor would use data APIs to continuously monitor and adjusts stock portfolios. 

This part of the exercise demonstrates:

1. Setting up a programming environment to work with APIs
2. Data preprocessing to create compatible datasets
3. Using a quantititive popular portfolio optimization library to estimate risks and weights

Documentation links:
<br />Alpha Vantage: https://www.alphavantage.co/documentation/
<br />Alpha Vantage Python library: https://alpha-vantage.readthedocs.io/en/latest/#
<br />Pandas DataFrame: https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
<br />PyPortfolioOpt: https://pyportfolioopt.readthedocs.io/en/latest/index.html

In [4]:
api_key = "OJTPVJGKW1IYYLF8"

In [5]:

import pandas as pd
import requests
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

# PyPortfolioOpt libraries
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [6]:
ts = TimeSeries(key=api_key,output_format='pandas')

In [7]:
TSN_data, TSN_meta_data = ts.get_daily(symbol='TSN', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

Isolate "close" column of data for each security:

In [8]:
TSN_clean = TSN_data['4. close']
msft_clean = msft_data['4. close']
aapl_clean = aapl_data['4. close']
amzn_clean = amzn_data['4. close']

Create new dataframe with tickers as columns:

In [9]:
df = pd.DataFrame(columns=['TSN', 'msft', 'aapl', 'amzn'])

Place data into new dataframe:

In [30]:
df['TSN'] = TSN_clean
df['msft'] = msft_clean
df['aapl'] = aapl_clean
df['amzn'] = amzn_clean
print(df)

               TSN     msft     aapl       amzn
date                                           
1999-10-19  15.875   86.313   68.500    76.6250
1999-10-20  15.750   92.250   75.125    80.5000
1999-10-21  15.750   93.063   76.125    80.7500
1999-10-22  15.875   92.688   73.938    78.6250
1999-10-25  15.750   92.438   74.500    82.7500
1999-10-26  15.625   92.375   75.063    81.2500
1999-10-27  15.375   90.875   76.375    75.9380
1999-10-28  15.313   89.875   77.875    71.0000
1999-10-29  15.250   92.563   80.125    70.6250
1999-11-01  15.750   92.375   77.625    69.1250
1999-11-02  15.875   92.563   80.250    66.4380
1999-11-03  16.250   92.000   81.500    65.8130
1999-11-04  16.375   91.750   83.625    63.0630
1999-11-05  16.688   91.563   88.313    64.9380
1999-11-08  16.500   89.938   96.375    78.0000
1999-11-09  16.438   88.875   89.625    70.8130
1999-11-10  16.875   87.125   91.438    72.0000
1999-11-11  17.250   89.625   92.250    73.0000
1999-11-12  17.438   89.188   90.625    

Take only the earlist half of observations:

In [31]:
df = df[0:924]

Calculate expected returns:

In [32]:
# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)

Calculate sample covariance matrix:

In [33]:
S = risk_models.sample_cov(df)

Optimize the weights to maximzie the Sharpe Ratio:

In [34]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 20.2%
Annual volatility: 92.2%
Sharpe Ratio: 0.20


(0.20240734040050548, 0.9215818484119076, 0.19792852985856252)

In [35]:
print(weights)

{'TSN': 9.71445146547012e-17, 'msft': 6.938893903907228e-18, 'aapl': 5.551115123125783e-17, 'amzn': 1.0}


Using optimized weights, compute portfolio accolation:

In [36]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

3 out of 4 tickers were removed
{'amzn': 28248}


To update portfolio, call the API again for latest data and run optimization model again:

In [37]:
TSN_data, TSN_meta_data = ts.get_daily(symbol='TSN', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

In [23]:
df = pd.DataFrame(columns=['TSN', 'msft', 'aapl', 'amzn'])

In [39]:
df['TSN'] = TSN_clean
df['msft'] = msft_clean
df['aapl'] = aapl_clean
df['amzn'] = amzn_clean
print(df)

               TSN    msft    aapl    amzn
date                                      
1999-10-19  15.875  86.313  68.500  76.625
1999-10-20  15.750  92.250  75.125  80.500
1999-10-21  15.750  93.063  76.125  80.750
1999-10-22  15.875  92.688  73.938  78.625
1999-10-25  15.750  92.438  74.500  82.750
1999-10-26  15.625  92.375  75.063  81.250
1999-10-27  15.375  90.875  76.375  75.938
1999-10-28  15.313  89.875  77.875  71.000
1999-10-29  15.250  92.563  80.125  70.625
1999-11-01  15.750  92.375  77.625  69.125
1999-11-02  15.875  92.563  80.250  66.438
1999-11-03  16.250  92.000  81.500  65.813
1999-11-04  16.375  91.750  83.625  63.063
1999-11-05  16.688  91.563  88.313  64.938
1999-11-08  16.500  89.938  96.375  78.000
1999-11-09  16.438  88.875  89.625  70.813
1999-11-10  16.875  87.125  91.438  72.000
1999-11-11  17.250  89.625  92.250  73.000
1999-11-12  17.438  89.188  90.625  74.938
1999-11-15  17.438  87.000  89.438  73.500
1999-11-16  17.250  87.313  91.188  78.938
1999-11-17 

In [40]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)
# Optimise for maximal Sharpe ratio"
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)
print(weights)

Expected annual return: 20.2%
Annual volatility: 92.2%
Sharpe Ratio: 0.20
{'TSN': 9.71445146547012e-17, 'msft': 6.938893903907228e-18, 'aapl': 5.551115123125783e-17, 'amzn': 1.0}


In [41]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

3 out of 4 tickers were removed
{'amzn': 28248}
